<a href="https://colab.research.google.com/github/yijunhou/CS229_Project/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from tqdm import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split


In [29]:
train_data = np.load('/content/gdrive/My Drive/CS229/project/Data/musicnet.npz', encoding='bytes',allow_pickle=True)

In [30]:
composer_lookup = {"Beethoven":0,"Schubert":1,"Brahms":2,"Mozart":3,"Bach":4,"Dvorak":5,"Cambini":6,"Faure":7,"Ravel":8,"Haydn":9}

In [31]:
df = pd.read_csv('/content/gdrive/My Drive/CS229/project/Data/data_file.csv')
df

,Unnamed: 0,ID,key_id,piece_id,composer,composer_id
0,0,0,1788,1,Mozart,3
1,1,1,1788,2,Mozart,3
2,2,2,1788,3,Mozart,3
3,3,3,1788,4,Mozart,3
4,4,4,1788,5,Mozart,3
...,...,...,...,...,...,...
12121,12121,12121,1765,63,Schubert,1
12122,12122,12122,1765,64,Schubert,1
12123,12123,12123,1765,65,Schubert,1
12124,12124,12124,1765,66,Schubert,1


In [7]:
fs = 44100
chunk_size = 10
sr = 10
n_classes = 10

In [34]:
i= 0
X, y = [], []
for data in tqdm(df.iterrows(),  desc='Progress'):
    key = str(data[1]['key_id'])
    piece = data[1]['piece_id'] -1
    chunks = int((len(train_data[key][0]))/fs/chunk_size)
    a = np.array_split(train_data[key][0], chunks)[piece][::sr]
    # mfcc_ = librosa.feature.mfcc(a, sr=fs, n_mfcc=40)
    X = np.append(X, a[0:400])
    del(a)
    y.append(data[1]['composer_id'])
    if i == 5000:
      break
    else:
      i += 1
    '''
    elif i % 10 == 0 and i > 0:
      X = np.array(X)
      np.save('batch_1.npy', X, allow_pickle = True, fix_imports=True)
      del(X)
      X = np.load('batch_1.npy', allow_pickle= True)
      i += 1
    '''
    

Progress: 5000it [3:08:23,  2.26s/it]


In [35]:
np.save('/content/gdrive/My Drive/CS229/project/Data/transformer_x.npy', X)
np.save('/content/gdrive/My Drive/CS229/project/Data/transformer_y.npy', y)

In [3]:
X = np.load('/content/gdrive/My Drive/CS229/project/Data/transformer_x.npy')
y = np.load('/content/gdrive/My Drive/CS229/project/Data/transformer_y.npy')

In [4]:
print(Counter(y))

Counter({0: 2345, 2: 777, 1: 614, 3: 542, 4: 407, 8: 134, 6: 81, 5: 79, 9: 22})


In [8]:
X_ = np.array(X)
print(X_.shape)
X_ = X_.reshape((5001, -1, 1))
y_ = tf.keras.utils.to_categorical(y , num_classes= n_classes)
print(X_.shape, y_.shape)

(2000400,)
(5001, 400, 1) (5001, 10)


In [9]:
x_train, x_rest, y_train, y_rest = train_test_split(X_, y_, test_size = 0.2, 
                                                   random_state = 42, stratify = y_)
x_validation, x_test, y_validation, y_test = train_test_split(x_rest, y_rest, test_size = 0.5, 
                                                   random_state = 42, stratify = y_rest)
print(x_train.shape, x_validation.shape, x_test.shape)

(4000, 400, 1) (500, 400, 1) (501, 400, 1)


In [10]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [11]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [12]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["acc"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    #validation_split=0.2,
    validation_data = (x_validation, y_validation),
    epochs=10,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400, 1)]     0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 400, 1)      2           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 400, 1)      7169        ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

[1.6049537658691406, 0.46906188130378723]

In [14]:
Counter(np.argmax(model.predict(x_train), axis=1))

Counter({0: 3999, 2: 1})